In [1]:
%config IPCompleter.greedy=True

In [91]:
from datetime import datetime
from models.data_provider import DataModel, adjust_and_log
import backtrader as bt
import heapq
# import tkinter
# import matplotlib
# matplotlib.use("tkagg")
# matplotlib.use('TKAgg')
# import matplotlib.pyplot as plt
# %matplotlib inline

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.order_target_size(target=1)  # enter long
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.order_target_size(target=0)  # close long position
            self.close()  # close long position

            
class StairCase(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )
    def __init__(self):
        self.day_ind = 0
        self.stock_heap = []
#         heapq.heappush(self.ip_q, (priority, ip))
#         heapq.heapify(self.ip_q)
#         heapq.heappop(self.ip_q)
        self.stochastic = bt.indicators.StochasticSlow(self.datas[0])
        self.macd = bt.indicators.MACDHisto(self.datas[0])
        self.rsi = bt.indicators.RSI(self.datas[0])

    def __buy(self, amount=1):
        pass
    
    def __sell(self, amount=1):
        pass
        
        
    def next(self):
        self.day_ind += 1
        if self.day_ind < 10:
            self.buy()
        if self.day_ind in range(10, 20):
            self.sell()
        if self.day_ind in range(20, 30):
            self.buy()
#         print(f"{self.position}")
#         print(f"----------------------------: {self.broker.cash}")
#         print(f"----------------------------: {self.broker.getvalue()}")
        
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f"{dt.isoformat()} {txt}")


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def stop(self):
        self.log(f'{self.broker.getvalue()}', doprint=True)

In [1]:
from models.data_provider import DataModel
import pandas as pd
dm2 = DataModel()
# dm2.read_from_csvs("./excels",["master0.csv","master1.csv"])
# df = pd.read_csv("./excels/master0.csv")
dm2.restore_from_pystore()
# dm2.save_to_csvs("./excels", "master", 1000000)

In [2]:
dm2.df["date"] = dm2.df.index
dm2.df["index_date"] = dm2.df.index
dm2.df["index_symbol"] = dm2.df.symbol
df = dm2.df.set_index(["index_symbol", "index_date"])
df["symbol"] = df.symbol.apply(str)

In [3]:
%%time
print(df[df["symbol"] == None])
err = []
for name in df["symbol"].unique():
    if name != "nan" and name != None:
#         print(name)
        try:
            df.loc[name]
        except:
            err.append(name)
    

Empty DataFrame
Columns: [symbol, name, amount, volume, value, lastday, open, close, last-change, last-percent, ending, ending-change, ending-percent, min, max, year, month, day, diff_min_max, diff_open, adj_min, adj_max, adj_close, adj_open, adj_ending, log_adj_open, log_adj_close, log_adj_ending, log_adj_min, log_adj_max, adj_scale, date]
Index: []

[0 rows x 32 columns]
CPU times: user 29.3 s, sys: 64.9 ms, total: 29.4 s
Wall time: 29.4 s


In [6]:
df[df["symbol"]=='None']

symbol  name  amount      volume         value  \
index_symbol index_date                                                  
NaN          2011-08-23   None  None    18.0    106305.0  3.818900e+08   
             2011-08-24   None  None     5.0  29022198.0  7.255550e+10   
             2011-08-27   None  None     4.0  29000000.0  7.250000e+10   
             2011-08-29   None  None     3.0     10000.0  2.400000e+07   
             2011-09-03   None  None     2.0     10000.0  2.400000e+07   
             2011-09-04   None  None     2.0     10000.0  2.400000e+07   
             2011-09-05   None  None     1.0     10000.0  2.400000e+07   
             2011-09-07   None  None     2.0     10000.0  2.400000e+07   
             2011-09-10   None  None     1.0     10000.0  2.400000e+07   
             2011-09-12   None  None     1.0     10000.0  3.400000e+07   
             2011-09-14   None  None    55.0      7975.0  1.914000e+07   
             2011-09-19   None  None    65.0      9425.0  2.262000e+07   
             2011-09-25   None  None     5.0       725.0  1.740000e+06   
             2011-09-26   None  None   237.0     33640.0  8.073600e+07   
             2011-09-27   None  None   108.0     15225.0  3.654000e+07   
             2011-10-01   None  None    23.0    114595.0  2.750280e+08   
             2011-10-10   None  None     6.0     22000.0  5.280000e+07   
             2011-10-17   None  None    29.0     70000.0  1.680000e+08   
             2011-10-18   None  None     2.0      6000.0  1.440000e+07   
             2011-10-19   None  None     1.0   4500000.0  4.990500e+10   
             2011-10-25   None  None     4.0      6000.0  1.440000e+07   
             2011-10-31   None  None     1.0       145.0  3.480000e+05   
             2011-11-05   None  None     4.0       580.0  1.392000e+06   
             2011-11-06   None  None     2.0      3145.0  7.548000e+06   
             2011-11-08   None  None     3.0       290.0  6.960000e+05   
             2012-01-03   None  None   544.0    837747.0  8.377470e+08   

                         lastday     open    close  last-change  last-percent  \
index_symbol index_date                                                         
NaN          2011-08-23   1000.0   3333.0   4000.0       3000.0        300.00   
             2011-08-24   1000.0   2500.0   2500.0       1500.0        150.00   
             2011-08-27   2500.0   2500.0   2500.0          0.0          0.00   
             2011-08-29   2500.0   2400.0   2400.0       -100.0         -4.00   
             2011-09-03   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-04   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-05   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-07   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-10   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-12   3428.0   3400.0   3400.0        -28.0         -0.82   
             2011-09-14   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-19   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-25   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-26   2400.0   2400.0   2400.0          0.0          0.00   
             2011-09-27   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-01   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-10   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-17   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-18   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-19   1000.0  11090.0  11090.0      10090.0       1009.00   
             2011-10-25   2400.0   2400.0   2400.0          0.0          0.00   
             2011-10-31   2400.0   2400.0   2400.0          0.0          0.00   
       

In [98]:
dm = DataModel()
dm.read_from_csvs("../xcels",["master0.csv","master1.csv"])

In [94]:
cerebro = bt.Cerebro()
# cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.broker.setcommission(commission=0.015)
cerebro.broker.setcash(10000000)
cerebro.addwriter(bt.WriterFile, out="celeb.log", seplen=150, close_out=True, csv=True)

df = dm.get(dm.TA_SYMBOLS[1],"1390-01-01")
df.columns = ['symbol', 'name', 'amount', 'volume', 'value', 'lastday', 'open1', 'close1', 'last-change', 'last-percent', 'ending', 'ending-change',
       'ending-percent', 'min', 'max', 'year', 'month', 'day', 'diff_min_max', 'diff_open', 'low', 'high', 'close', 'open',
       'adj_ending', 'log_adj_open', 'log_adj_close', 'log_adj_ending', 'log_adj_min', 'log_adj_max', 'adj_scale']
data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)

# cerebro.addstrategy(StairCase)
strats = cerebro.addstrategy(StairCase, maperiod=range(10))
cerebro.run()
# b = Bokeh(style='bar', plot_mode='single')
# chart = cerebro.plot(style='candlestick')
chart = cerebro.plot(iplot=False, numfigs=1, width=200, height=200, dpi=500, tight=True)
chart[0][0].savefig('cele2.jpg', dpi=150)
# chart[0][0].show()

TypeError: cannot serialize '_io.TextIOWrapper' object

In [8]:
import pandas as pd
df0 = dm.get(dm.TA_SYMBOLS[1],"1390-01-01")
df1 = dm.get(dm.TA_SYMBOLS[5],"1390-01-01")
df = pd.concat([df0, df1])
df.groupby("symbol").apply(lambda x: print(x))

2020-01-09 21:47:58.187 | DEBUG    | models.data_provider:get:172 - symbol is not scaled: كنور
2020-01-09 21:47:58.191 | DEBUG    | models.data_provider:adjust_and_log:57 - start adjust and log for كنور
2020-01-09 21:47:58.192 | DEBUG    | models.data_provider:adjust_and_log:58 - calculating scale
2020-01-09 21:47:58.206 | DEBUG    | models.data_provider:adjust_and_log:60 - adding adjust
2020-01-09 21:47:58.208 | DEBUG    | models.data_provider:add_adjust:28 - new part len is 1640, shape: (1640,)
2020-01-09 21:47:58.235 | DEBUG    | models.data_provider:add_adjust:38 - found adj date: 2013-06-08 00:00:00
2020-01-09 21:47:58.242 | DEBUG    | models.data_provider:add_adjust:38 - found adj date: 2014-05-06 00:00:00
2020-01-09 21:47:58.250 | DEBUG    | models.data_provider:add_adjust:38 - found adj date: 2014-06-01 00:00:00
2020-01-09 21:47:58.259 | DEBUG    | models.data_provider:add_adjust:38 - found adj date: 2014-07-20 00:00:00
2020-01-09 21:47:58.267 | DEBUG    | models.data_provider:

           symbol         name  amount      volume         value  lastday  \
date                                                                        
2011-03-26  خكاوه  سايپا ديزل‌    41.0    274146.0  2.570260e+08    849.0   
2011-03-27  خكاوه  سايپا ديزل‌    73.0   1051351.0  1.024442e+09    938.0   
2011-03-28  خكاوه  سايپا ديزل‌    60.0    628797.0  6.196478e+08    974.0   
2011-03-29  خكاوه  سايپا ديزل‌    25.0    122576.0  1.201572e+08    985.0   
2011-03-30  خكاوه  سايپا ديزل‌    51.0    460431.0  4.365755e+08    984.0   
2011-04-03  خكاوه  سايپا ديزل‌   173.0    868288.0  8.545096e+08    950.0   
2011-04-04  خكاوه  سايپا ديزل‌   211.0   3507791.0  3.588470e+09    984.0   
2011-04-05  خكاوه  سايپا ديزل‌   427.0   5956898.0  6.330631e+09   1023.0   
2011-04-06  خكاوه  سايپا ديزل‌   159.0   1078255.0  1.143680e+09   1063.0   
2011-04-09  خكاوه  سايپا ديزل‌    42.0    334879.0  3.423204e+08   1061.0   
2011-04-10  خكاوه  سايپا ديزل‌    19.0     90056.0  8.942561e+07   1034.0   

""


In [ ]:

from loguru import logger
logger.remove()
logger.add("log/test-data_crawler_debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True, backtrace=True)
logger.add("log/test-data_crawler_info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
logger.add("log/test-data_crawler_error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)
gdf = dm.df.groupby("symbol").apply(adjust_and_log)

In [ ]:
gdf

In [22]:
lens[dm.TA_SYMBOLS[1]]

3168

In [ ]:
import backtrader as bt

# Create a subclass of SignaStrategy to define the indicators and signals

class SmaCross(bt.SignalStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.signal_add(bt.SIGNAL_LONG, crossover)  # use it as LONG signal


cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed
data = bt.feeds.YahooFinanceData(dataname='MSFT',
                                 fromdate=datetime(2011, 1, 1),
                                 todate=datetime(2012, 12, 31))

cerebro.adddata(data)  # Add the data feed

cerebro.addstrategy(SmaCross)  # Add the trading strategy
cerebro.run()  # run it all
cerebro.plot()  # and plot it with a single command